In [1]:
import pandas as pd
import numpy as np
from biopsy_grade_extractor import BiopsyGradeExtractor # load exttractor

### load the dataset which contrains biopsy reports

In [2]:
df_pathology = pd.read_csv('../data/sample_biopsy_reports.csv')	

In [3]:
df_pathology

,Unnamed: 0,EMPI,MRN_Type,Report_Number,Report_Text
0,0,5162687835,MGH,YDOXSRZFJF,Firm right prostate lobe. ====================...
1,1,6545694958,MGH,PWDMXHIVGX,"A. PROSTATE NEEDLE BIOPSIES, RIGHT BASE: PROST..."
2,2,7804781214,BWH,KFDPJIOOSD,A) RIGHT LOBE: Benign prostatic tissue. Immuno...
3,3,480085707,BWH,KKOAKIFZSY,PROSTATE; NEEDLE BIOPSIES : A) LEFT SIDE; PROS...
4,4,6853020993,BWH,FMPNJLHJYW,"RIGHT PROSTATE, NEEDLE BIOPSY: PROSTATIC ADENO..."
5,5,7042372336,BWH,HQZQSVIBQE,A) LEFT APEX: Benign prostatic tissue. B) LEFT...
6,6,1134543123,BWH,TCHTAECHHT,CORE NEEDLE BIOPSIES (XXX): A) RIGHT APEX: Ben...
7,7,4022503058,MGH,MSWIDQDSFI,"A. PROSTATE, RIGHT BASE: PROSTATIC ADENOCARCIN..."
8,8,2586526426,BWH,SMOEUKNPBZ,A. RIGHT APEX (BIOPSY/IES): Atypical small aci...
9,9,4954105822,MGH,DGPIVXRPDB,"A. PROSTATE, RIGHT 4 MID (S13-09075-H; 02/27/2..."


### Instantiate BiopsyGradeExtractor and fit the data

In [4]:
extractor = BiopsyGradeExtractor()
extractor.fit(df_pathology, display_summary = False, return_summary_df = True)

extracing info...: 100%|██████████| 18/18 [00:00<00:00, 4053.12it/s]



Extraction complete.


Displaying summary...
Counts per each primary grade: 
3    30
4    11
5     3
dtype: int64


Counts per each secondary grade: 
3    25
4    13
5     6
dtype: int64


Counts per each max core involvement : 
5     10
10     8
30     6
20     6
15     5
90     4
50     4
25     3
60     2
70     2
40     2
17     1
80     1
2      1
dtype: int64


Number of reports with benign indicator 1 (postiive) or 0 (negative) : 
0.0    18
dtype: int64
total =  18


Number of reports per each # of positive cores : 
1.0     5
2.0     4
4.0     3
3.0     2
9.0     1
10.0    1
5.0     1
8.0     1
dtype: int64
Number of reports per each # of total cores : 
4.0     4
6.0     4
2.0     3
10.0    2
1.0     2
12.0    1
3.0     1
5.0     1
dtype: int64
Number of reports per each overall grade : 
1.0    7
3.0    5
4.0    2
5.0    2
2.0    2
dtype: int64
total =  18
Number of unique biopsy reports :  18
Number of unique biopsy reports per center : 
BWH    13
MGH     5
dtype: int64
Numbe


/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/missing.py:531: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.array(val, copy=False)
/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarr

,Unnamed: 0,EMPI,MRN_Type,Report_Number,Report_Text,num_pos_cores,primary_grade,secondary_grade,overall_grade_group,overall_gs,overall_grade_merged,benign,num_total_core,num_pos_cores_sum,num_total_core_sum,max_core_involve,small_cell_carc,neuroendocrine_carc,adenocarcinoma
0,0,5162687835,MGH,YDOXSRZFJF,Firm right prostate lobe. ====================...,[4],[3],[3],[1],[6],1.0,0.0,[4],4.0,4.0,[[60]],0.0,0.0,1.0
1,1,6545694958,MGH,PWDMXHIVGX,"A. PROSTATE NEEDLE BIOPSIES, RIGHT BASE: PROST...","[1, 2, 2, 1, 1, 2]","[4, 4, 4, 3, 3, 3]","[4, 4, 3, 4, 4, 3]","[4, 4, 3, 2, 2, 1]","[8, 8, 7, 7, 7, 6]",4.0,0.0,"[2, 2, 2, 2, 2, 2]",9.0,12.0,"[[10], [30, 10], [90, 30], [5], [5], [20, 5]]",0.0,0.0,1.0
2,2,7804781214,BWH,KFDPJIOOSD,A) RIGHT LOBE: Benign prostatic tissue. Immuno...,[4],[4],[3],[3],[7],3.0,0.0,[6],4.0,6.0,"[[90, 70, 20, 20]]",0.0,0.0,1.0
3,3,480085707,BWH,KKOAKIFZSY,PROSTATE; NEEDLE BIOPSIES : A) LEFT SIDE; PROS...,"[5, 5]","[3, 3]","[3, 3]","[1, 1]","[6, 6]",1.0,0.0,"[5, 5]",10.0,10.0,"[[60, 50, 50, 50, 30], [30, 30, 25, 20, 5]]",0.0,0.0,1.0
4,4,6853020993,BWH,FMPNJLHJYW,"RIGHT PROSTATE, NEEDLE BIOPSY: PROSTATIC ADENO...","[2, 3]","[5, 5]","[5, 5]","[5, 5]","[10, 10]",5.0,0.0,"[3, 3]",5.0,6.0,"[[40], [15]]",0.0,0.0,1.0
5,5,7042372336,BWH,HQZQSVIBQE,A) LEFT APEX: Benign prostatic tissue. B) LEFT...,"[1, 1]","[4, 3, 3, 3]","[3, 4, 3, 4]","[3, 2, 1, 2]","[7, 7, 6, 7]",3.0,0.0,"[3, 3]",2.0,6.0,"[[10], [10]]",0.0,0.0,1.0
6,6,1134543123,BWH,TCHTAECHHT,CORE NEEDLE BIOPSIES (XXX): A) RIGHT APEX: Ben...,"[2, 2]","[3, 3]","[3, 4]","[1, 2]","[6, 7]",2.0,0.0,"[2, 2]",4.0,4.0,"[[25, 5], [5, 10]]",0.0,0.0,1.0
7,7,4022503058,MGH,MSWIDQDSFI,"A. PROSTATE, RIGHT BASE: PROSTATIC ADENOCARCIN...",[1],[3],[3],[1],[6],1.0,0.0,[2],1.0,2.0,[[5]],0.0,0.0,1.0
8,8,2586526426,BWH,SMOEUKNPBZ,A. RIGHT APEX (BIOPSY/IES): Atypical small aci...,"[1, 1]","[4, 3, 4]","[3, 4, 3]","[3, 2, 3]","[7, 7, 7]",3.0,0.0,"[2, 2]",2.0,4.0,"[[20], [15]]",0.0,0.0,1.0
9,9,4954105822,MGH,DGPIVXRPDB,"A. PROSTATE, RIGHT 4 MID (S13-09075-H; 02/27/2...","[1, 1, 1]","[3, 3, 3]","[3, 3, 3]","[1, 1, 1]","[6, 6, 6]",1.0,0.0,"[2, 2, 2]",3.0,6.0,"[[30], [10], [5]]",0.0,0.0,1.0


In [7]:
df_pathology.iloc[1]['Report_Text']

'A. PROSTATE NEEDLE BIOPSIES, RIGHT BASE: PROSTATIC ADENOCARCINOMA, Gleason score 4+4=8 (Grade Group 4) involving 10% of one of two (1:2) fragmented cores. No perineural invasion. B. PROSTATE NEEDLE BIOPSIES, RIGHT MID: PROSTATIC ADENOCARCINOMA, Gleason score 4+4=8 (Grade Group 4) involving 30% and 10% of two of two (2:2) fragmented cores. No perineural invasion. C. PROSTATE NEEDLE BIOPSIES, RIGHT APEX: PROSTATIC ADENOCARCINOMA, Gleason score 4+3=7 (Grade Group 3) involving 90% and 30% of two of two (2:2) fragmented cores. Gleason pattern 4 comprises 70% of the tumor. Perineural invasion is present. D. PROSTATE NEEDLE BIOPSIES, LEFT BASE: PROSTATIC ADENOCARCINOMA, Gleason score 3+4=7 (Grade Group 2) involving less than 5% of one of two (1:2) cores. Gleason pattern 4 comprises 20% of the tumor. No perineural invasion. E. PROSTATE NEEDLE BIOPSIES, LEFT MID: PROSTATIC ADENOCARCINOMA, Gleason score 3+4=7 (Grade Group 2) involving 5% of one of two (1:2) cores. Gleason pattern 4 comprises 30

### Validate the performance of the algorithm

In [9]:
# if report_number_oi is not specified, then it randomly picks report to evaluate
for i in range(2):
    extractor.validate_results()



Extracted info : 
Report_Number                                                  TCHTAECHHT
Report_Text             CORE NEEDLE BIOPSIES (XXX): A) RIGHT APEX: Ben...
primary_grade                                                      [3, 3]
secondary_grade                                                    [3, 4]
overall_grade_group                                                [1, 2]
overall_grade_merged                                                  2.0
Name: 6, dtype: object


Pathology report : 
CORE NEEDLE BIOPSIES (XXX): A) RIGHT APEX: Benign prostatic tissue. B) RIGHT MID: PROSTATIC ADENOCARCINOMA, Gleason score 3+3=6, involving 25% and 5% of two of two cores (2:2). No perineural invasion. C) RIGHT BASE: PROSTATIC ADENOCARCINOMA, Gleason score 3+4=7, involving 5% and 10% of two of two cores (2:2). No perineural invasion. D) LEFT APEX: Benign prostatic tissue. E) LEFT MID: Benign prostatic tissue. F) LEFT BASE: Benign prostatic tissue. 




Extracted info : 
Report_Number    

In [ ]:
# Primary Grade/Secondary Grade/overall grade merged